In [2]:
from mads_datasets.base import BaseDatastreamer
from mltrainer.preprocessors import BasePreprocessor
from pathlib import Path
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
from torch import nn
import torch
import numpy as np
import tomllib

from mads_hackathon import datasets, metrics


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/svelthuis/group_c/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/svelthuis/group_c/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/svelthuis/group_c/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start(

First, go to the `config.toml` file and change the `dev` value to your name. Please use `firstname_lastname` format. 
In addition to that, change the `port` value to the port of your team.

In [3]:

configfile = Path("config.toml")
with configfile.open("rb") as f:
    tomlconfig = tomllib.load(f)

assert tomlconfig["dev"] != "dev", ValueError("Please set dev in config.toml to your own name")
assert tomlconfig["port"] != "none", ValueError("Please set port in config.toml to your own port")
uri = tomlconfig["mlflow_uri"] + ":" + tomlconfig["port"]
dev = tomlconfig["dev"]
print(f"Using {uri} as mlfow uri")
print(f"Using {dev} as dev name")

Using http://145.38.195.42:5002 as mlfow uri
Using alex as dev name


Load the datasets

In [4]:
datadir = Path('../hackathon-data/')
trainfile = (datadir / "heart_big_train.parq").resolve()
validfile = (datadir / "heart_big_valid.parq").resolve()
trainfile.exists(), validfile.exists()

(True, True)

Change the backend, if you have one.

In [ ]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

using cuda


Change the 1D data into a 2D matrix with the HeartDataset2D class. See `src/mads_hackathon/datasets.py` for more information.

In [10]:
matrixshape = (3, 48)
traindataset = datasets.HeartDataset2D_over(trainfile, target="target", shape=matrixshape)
testdataset = datasets.HeartDataset2D(validfile, target="target", shape=matrixshape)
traindataset.to(device)
testdataset.to(device)


Let's see how that works. 
Compare this to the 1D data from notebook `01_explore-heart.ipynb`

In [7]:
x, y = traindataset[0]
print(len(x))
# Assuming your tensor is named 'tensor'
viz = x.squeeze().cpu().numpy()
sns.heatmap(viz, cmap='rainbow')
x.shape, y

1


RuntimeError: Numpy is not available

First, we set the config, then load into a streamer

In [29]:
from mads_hackathon.models import CNNConfig as Config

config = Config(
    matrixshape = (4,48),
    batchsize = 64,
    input_channels = 1,
    hidden = 16,
    kernel_size = 3,
    maxpool = 2,
    num_layers = 1,
    num_classes = 5,
)
config

CNNConfig(matrixshape=(4, 48), batchsize=64, input_channels=1, hidden=16, kernel_size=3, maxpool=2, num_layers=1, num_classes=5)

In [30]:
trainstreamer = BaseDatastreamer(traindataset, preprocessor = BasePreprocessor(), batchsize=config.batchsize)
teststreamer = BaseDatastreamer(testdataset, preprocessor = BasePreprocessor(), batchsize=config.batchsize)
len(trainstreamer), len(teststreamer)

(1368, 171)

Let's check the shape

In [31]:
x, y = next(trainstreamer.stream())
x.shape, y.shape

(torch.Size([64, 1, 4, 48]), torch.Size([64]))

We can load the CNN model. It uses a ConvBlock class that is a wrapper around a Conv2D and ReLU stack.
The ConvBlock makes it easier to stack block in the CNN model, see `src/mads_hackathon/models.py` for the implementation.

This is just a initial setup: please experiment with other architectures like:
- adding ideas like resnet, googlenet, squeeze-excite, etc.
- add additional layers like dropout, batchnorm, etc.
- experiment with different ways to go from 4D to 2D tensors

In [32]:
from mads_hackathon.models import CNN
model = CNN(config)

Calculated matrix size: 48
Caluclated flatten size: 768


Let's check the architecture

In [33]:
from torchinfo import summary
summary(model, input_size=(config.batchsize, 1, *config.matrixshape))

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [64, 5]                   --
├─ModuleList: 1-1                        --                        --
│    └─ConvBlock: 2-1                    [64, 16, 4, 48]           --
│    │    └─Sequential: 3-1              [64, 16, 4, 48]           2,480
│    └─ConvBlock: 2-2                    [64, 16, 4, 48]           --
│    │    └─Sequential: 3-2              [64, 16, 4, 48]           4,640
│    └─ReLU: 2-3                         [64, 16, 4, 48]           --
│    └─MaxPool2d: 2-4                    [64, 16, 2, 24]           --
├─Sequential: 1-2                        [64, 5]                   --
│    └─Flatten: 2-5                      [64, 768]                 --
│    └─Linear: 2-6                       [64, 16]                  12,304
│    └─ReLU: 2-7                         [64, 16]                  --
│    └─Linear: 2-8                       [64, 5]                   85
Total

And test if the model works

In [34]:
model.to(device)
yhat = model(x)
yhat.shape

torch.Size([64, 5])

The data is unbalanced, so we are interested in much more than just accuracy.
See https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html for more information on the F1 micro/macro score.
See `src/mads_hackathon/metrics.py` for the implementation. You might want to add more metrics, or change the settings for `average`

In [35]:
f1micro = metrics.F1Score(average='micro')
f1macro = metrics.F1Score(average='macro')
precision = metrics.Precision(average='macro')
recall = metrics.Recall('macro')
accuracy = metrics.Accuracy()
print(f1micro)

F1scoremicro


We also want a confusion matrix to see how the model is performing on the different classes.

In [36]:
from mads_hackathon.metrics import caluclate_cfm

We set the mlflow server to log the results

In [37]:
import mlflow
mlflow.set_tracking_uri(uri)
mlflow.set_experiment("test1-alex")

2024/11/25 12:37:25 INFO mlflow.tracking.fluent: Experiment with name 'test1-alex' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/703179046667895742', creation_time=1732534645878, experiment_id='703179046667895742', last_update_time=1732534645878, lifecycle_stage='active', name='test1-alex', tags={}>

In [38]:
config = Config(
    matrixshape = (4,48),
    batchsize = 64,
    input_channels = 1,
    hidden = 16,
    kernel_size = 3,
    maxpool = 2,
    num_layers = 1,
    num_classes = 5,
)
config

CNNConfig(matrixshape=(4, 48), batchsize=64, input_channels=1, hidden=16, kernel_size=3, maxpool=2, num_layers=1, num_classes=5)

Please dont just run the cell below, but walk trough what is being logged.
For example, you might want to change the tag for the model and the dataset if you make changes there, and 
to the scheduler/earlystop if you are using it.

In [56]:
import torch

train_df = pd.read_parquet(trainfile)
train_percentages = train_df.target.value_counts(normalize=True).sort_index() * 100

weights1 = torch.tensor((100 / train_percentages).to_numpy()).float().to(device)
weights2 = torch.tensor((1 / train_percentages).to_numpy()).float().to(device)

weights1, weights2


(tensor([  1.2081,  39.3855,  15.1268, 136.5897,  13.6144]),
 tensor([0.0121, 0.3939, 0.1513, 1.3659, 0.1361]))

In [43]:
from mltrainer import Trainer, TrainerSettings, ReportTypes
from dataclasses import asdict

loss_fn = torch.nn.CrossEntropyLoss(weight=weights1)

with mlflow.start_run():
    optimizer = torch.optim.Adam

    settings = TrainerSettings(
        epochs=5,
        metrics=[accuracy, f1micro, f1macro, precision, recall],
        logdir="logs/heart2D",
        train_steps=len(trainstreamer),
        valid_steps=len(teststreamer),
        reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
        scheduler_kwargs=None,
        earlystop_kwargs=None
    )

    # modify the tags when you change them!
    mlflow.set_tag("model", "CNN")
    mlflow.set_tag("dataset", "heart2D")
    mlflow.set_tag("dev", dev)
    mlflow.log_param("scheduler", "None")
    mlflow.log_param("earlystop", "None")

    mlflow.log_params(asdict(config))
    mlflow.log_param("epochs", settings.epochs)
    mlflow.log_param("matrix0", config.matrixshape[0])
    mlflow.log_param("matrix1", config.matrixshape[1])
    mlflow.log_param("optimizer", str(optimizer))
    mlflow.log_params(settings.optimizer_kwargs)

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optimizer,
        traindataloader=trainstreamer.stream(),
        validdataloader=teststreamer.stream(),
        scheduler=None,
        device=device,
        )
    trainer.loop()
    cfm = caluclate_cfm(model, teststreamer)
    for i, tp in enumerate(np.diag(cfm)):
        mlflow.log_metric(f"TP_{i}", tp)

plot = sns.heatmap(cfm, annot=cfm, fmt=".3f")
plot.set(xlabel="Predicted", ylabel="Target")

2024-11-25 12:39:16.301 | INFO     | mltrainer.settings:check_path:61 - Created logdir /Users/alex/Desktop/advancedai/hackathon/group_c/notebooks/logs/heart2D
2024-11-25 12:39:16.592 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to logs/heart2D/20241125-123916
  0%|          | 0/5 [00:00<?, ?it/s]


🏃 View run wise-jay-164 at: http://145.38.195.42:5002/#/experiments/703179046667895742/runs/4f6fcde442fb4f2f84d97b42a345a8f0
🧪 View experiment at: http://145.38.195.42:5002/#/experiments/703179046667895742


RuntimeError: Numpy is not available

Please note that, while the TP score for class 0 does get high without much effort, the challenge here will be in the other classes that lag behind.